In [14]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.preprocessing import sequence
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn.pipeline import Pipeline

In [2]:
df = pd.read_csv('../train.csv')
print(df.shape)
df.head()

(10886, 12)


,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


In [3]:
df_train = df.copy()
df_train['month'] = pd.DatetimeIndex(df_train['datetime']).month
# df_train = df[df['datetime'] < '2011-02'].copy()
# df_train = df[(df['datetime'] > month_ini) & (df['datetime'] < month_end)].copy()
df_train['day'] = pd.DatetimeIndex(df_train['datetime']).day
df_train['dayofweek'] = pd.DatetimeIndex(df_train['datetime']).dayofweek
df_train['hour'] = pd.DatetimeIndex(df_train['datetime']).hour
df_train = df_train.drop('datetime', axis=1)
y = df_train['count'].copy()
df_train = df_train.drop(['casual','registered','count'], axis=1)
# df = df.drop(['casual','registered'], axis=1)
print(df_train.shape)
df_train.head()

(10886, 12)


,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,month,day,dayofweek,hour
0,1,0,0,1,9.84,14.395,81,0.0,1,1,5,0
1,1,0,0,1,9.02,13.635,80,0.0,1,1,5,1
2,1,0,0,1,9.02,13.635,80,0.0,1,1,5,2
3,1,0,0,1,9.84,14.395,75,0.0,1,1,5,3
4,1,0,0,1,9.84,14.395,75,0.0,1,1,5,4


In [15]:
X = df_train.values
Y = y.values
maxlen = 20
X = sequence.pad_sequences(X, maxlen=maxlen)
def new_model():
    n_features = 12
    # create model
    model = Sequential()
    model.add(LSTM(50, input_dim=n_features, activation='relu'))
    model.add(Dense(1, init='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model
estimators = []
estimators.append(('scale', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=new_model, nb_epoch=100, batch_size=5, verbose=2)))
pipeline = Pipeline(estimators)
pipeline.fit(X, Y)
kfold = KFold(n_splits=7)
%time results = cross_val_score(pipeline, X, Y, cv=kfold, verbose=2)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

/home/st/anaconda3/envs/keras/lib/python3.5/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int32 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)


Exception: Error when checking model input: expected lstm_input_5 to have 3 dimensions, but got array with shape (10886, 20)

## Prediction

In [5]:
df_test = pd.read_csv('../test.csv')
df_test.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed
0,2011-01-20 00:00:00,1,0,1,1,10.66,11.365,56,26.0027
1,2011-01-20 01:00:00,1,0,1,1,10.66,13.635,56,0.0000
2,2011-01-20 02:00:00,1,0,1,1,10.66,13.635,56,0.0000
3,2011-01-20 03:00:00,1,0,1,1,10.66,12.880,56,11.0014
4,2011-01-20 04:00:00,1,0,1,1,10.66,12.880,56,11.0014


In [6]:
dft = df_test.copy()
dft['month'] = pd.DatetimeIndex(dft['datetime']).month
# dft = df_test[df_test['datetime'] < '2011-02'].copy()
# dft = df_test[(df_test['datetime'] > month_ini) & (df_test['datetime'] < month_end)].copy()
dft['day'] = pd.DatetimeIndex(dft['datetime']).day
dft['dayofweek'] = pd.DatetimeIndex(dft['datetime']).dayofweek
dft['hour'] = pd.DatetimeIndex(dft['datetime']).hour
dft = dft.drop('datetime', axis=1)
print(dft.shape)
dft.head()

(6493, 12)


,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,month,day,dayofweek,hour
0,1,0,1,1,10.66,11.365,56,26.0027,1,20,3,0
1,1,0,1,1,10.66,13.635,56,0.0000,1,20,3,1
2,1,0,1,1,10.66,13.635,56,0.0000,1,20,3,2
3,1,0,1,1,10.66,12.880,56,11.0014,1,20,3,3
4,1,0,1,1,10.66,12.880,56,11.0014,1,20,3,4


In [7]:
test_pred = pipeline.predict(dft)
test_pred[test_pred < 0] = 0

str_prediction = "datetime,count\n"
for i in range(0, len(test_pred)):
    datetime = df_test['datetime'][i]
    pred = int(round(test_pred[i]))
    str_prediction += "{},{}\n".format(datetime, pred)

# print(str_prediction)
f = open('pred_keras.csv','w')
f.write(str_prediction)
f.close()

# Month by month

In [29]:
def new_model():
    n_features = 11
    # create model
    model = Sequential()
    model.add(Dense(80, input_dim=n_features, init='normal', activation='relu'))
#     model.add(Dense(4, init='normal', activation='relu'))
    model.add(Dense(1, init='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [37]:
ranges = pd.date_range('2010-12-31 23:59:59', periods=25, freq='M').tolist()
ini = True
df_train = df.copy()
dft = df_test.copy()
df_train['datetime'] = pd.to_datetime(df_train['datetime'])
dft['datetime'] = pd.to_datetime(dft['datetime'])
for i in range(0, len(ranges)-1):
    print(ranges[i+1])
    df_month = df_train[(df_train['datetime'] > ranges[i]) & (df_train['datetime'] < ranges[i+1])].copy()
    df_month['day'] = pd.DatetimeIndex(df_month['datetime']).day
    df_month['dayofweek'] = pd.DatetimeIndex(df_month['datetime']).dayofweek
    df_month['hour'] = pd.DatetimeIndex(df_month['datetime']).hour
    Y = df_month['count'].values
    df_month = df_month.drop(['casual','registered','count','datetime'], axis=1)
    X = df_month.values
    estimators = []
    estimators.append(('scale', StandardScaler()))
    estimators.append(('mlp', KerasRegressor(build_fn=new_model, nb_epoch=100, batch_size=5, verbose=0)))
    pipeline = Pipeline(estimators)
    pipeline.fit(X, Y)
    dftm = dft[(dft['datetime'] > ranges[i]) & (dft['datetime'] < ranges[i+1])].copy()
    dftm['day'] = pd.DatetimeIndex(dftm['datetime']).day
    dftm['dayofweek'] = pd.DatetimeIndex(dftm['datetime']).dayofweek
    dftm['hour'] = pd.DatetimeIndex(dftm['datetime']).hour
    dates = dftm['datetime'].tolist()
    dftm = dftm.drop('datetime', axis=1)
    test_pred = pipeline.predict(dftm)
    test_pred[test_pred < 0] = 0
    str_prediction = ""
    if ini:
        str_prediction = "datetime,count\n"
        ini = False
    for j in range(0, len(test_pred)):
        datetime = dates[j]
        pred = int(round(test_pred[j]))
        str_prediction += "{},{}\n".format(datetime, pred)
    f = open('pred_keras2.csv','a')
    f.write(str_prediction)
    f.close()

2011-01-31 23:59:59
2011-02-28 23:59:59
2011-03-31 23:59:59
2011-04-30 23:59:59
2011-05-31 23:59:59
2011-06-30 23:59:59
2011-07-31 23:59:59
2011-08-31 23:59:59
2011-09-30 23:59:59
2011-10-31 23:59:59
2011-11-30 23:59:59
2011-12-31 23:59:59
2012-01-31 23:59:59
2012-02-29 23:59:59
2012-03-31 23:59:59
2012-04-30 23:59:59
2012-05-31 23:59:59
2012-06-30 23:59:59
2012-07-31 23:59:59
2012-08-31 23:59:59
2012-09-30 23:59:59
2012-10-31 23:59:59
2012-11-30 23:59:59
2012-12-31 23:59:59


In [36]:
ranges = pd.date_range('2010-12-31 23:59:59', periods=25, freq='M').tolist()
dftm = dft[(dft['datetime'] > ranges[1]) & (dft['datetime'] < ranges[2])].copy()
dates = dftm['datetime'].tolist()
dates[0]

Timestamp('2011-02-20 00:00:00')